## Initialization

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import matplotlib.pyplot as plt

import os
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

#Models
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor


# Model evaluation und visualisation
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

np.random.seed(42)

## Load and investigate data

In [ ]:
# NOTICE --- dtype={'Date': 'str'}
#data = pd.read_csv('beverage-import.csv',dtype={'Date': 'str'})

from google.colab import drive 
drive.mount("/content/gdrive")
data = pd.read_csv('/content/gdrive/MyDrive/Project/beverage-import.csv', index_col=0)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,13,14,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
column_names = data.columns
for c in column_names:
    print( c, ':', data[c].nunique(), data[c].isna().sum() )

Customer : 3112 0
Sales Rep : 45 0
Month : 12 0
Year : 3 0
Date : 28 0
Code and description  : 1136 0
Code Prod : 723 0
Description : 1328 0
Quantity : 1375 0
Units : 7 5767
Total quantity : 1264 0
Value Sale : 28467 0
Tax : 22865 0
Total value sale : 26529 0
Channel : 15 48
Brand : 62 0
Category : 14 0
Comment : 14 80505
Unit price : 9339 629
Unnamed: 20 : 0 81228


## Clean the data

In [ ]:
data = data.drop(['Unnamed: 20', 'Comment'], axis=1)

In [ ]:
data['NewDate']= data['Date'] \
        .map({'ENERO -- 2015':1, 'FEBRERO -- 2015':2, 'MARZO -- 2015':3,
       'ABRIL -- 2015':4, 'MAYO -- 2015':5, 'JUNIO -- 2015':6, 'JULIO -- 2015':7,
       'AGOSTO -- 2015':8, 'SEPTIEMBRE -- 2015':9, 'OCTUBRE -- 2015':10,
       'NOVIEMBRE -- 2015':11, 'DICIEMBRE -- 2015':12, 'ENERO -- 2016':13,
       'FEBRERO -- 2016':14, 'MARZO -- 2016':15, 'ABRIL -- 2016':16,
       'MAYO -- 2016':17, 'JUNIO -- 2016':18, 'JULIO -- 2016':19, 'AGOSTO -- 2016':20,
       'SEPTIEMBRE -- 2016':21, 'OCTUBRE -- 2016':22, 'NOVIEMBRE -- 2016':23,
       'DICIEMBRE -- 2016':24, 'ENERO -- 2017':25, 'FEBRERO -- 2017':26,
       'MARZO -- 2017':27, 'ABRIL -- 2017':28})

In [ ]:
# NewDate replaces them
data=data.drop(['Month','Year','Date'],axis=1)

In [ ]:
# NewDate is now numeric
numeric_columns=['NewDate',"Quantity", "Total quantity", "Value Sale", "Tax", "Total value sale", "Unit price" ]

In [ ]:
#Removing brackets
data['Total value sale'] = data['Total value sale'].replace(to_replace='\(', value="", regex=True)
data['Total value sale'] = data['Total value sale'].replace(to_replace='\)', value="", regex=True)

In [ ]:
data = data.dropna() # Removes every row which has at least one NaN

In [ ]:
for c in numeric_columns:
    data[c].replace(',', '', regex=True, inplace=True)
    data[c]= pd.to_numeric( data[c],errors='raise') 

In [ ]:
# Some more cleaning ... (used to be in the cat_names handling)
data=data.drop(['Description', 'Code and description '],axis=1)

In [ ]:
print('Data length ',len(data),'\n' )

column_names = data.columns
for c in column_names:
    print( c, ':',type(data[c][0]),data[c].nunique() )

Data length  74832 

Customer : <class 'str'> 3034
Sales Rep : <class 'str'> 45
Code Prod : <class 'str'> 715
Quantity : <class 'numpy.float64'> 896
Units : <class 'str'> 7
Total quantity : <class 'numpy.float64'> 896
Value Sale : <class 'numpy.float64'> 24043
Tax : <class 'numpy.float64'> 18371
Total value sale : <class 'numpy.float64'> 21633
Channel : <class 'str'> 14
Brand : <class 'str'> 62
Category : <class 'str'> 13
Unit price : <class 'numpy.float64'> 6921
NewDate : <class 'numpy.int64'> 28


In [ ]:
originalData = data

## Test-Train split and DataFrame to Numpy

In [ ]:
# Change -- first 25 vs. final 3 months

# The first 25 months are the training set and the final 3 months are the test set
# Other combinations are possible, e.g., more months at the end

features_train=data[data['NewDate']<23]
features_test=data[data['NewDate']>=23]

In [ ]:
label_train=features_train['Total value sale']
label_test=features_test['Total value sale']

In [ ]:
features_train=features_train.drop('Total value sale', axis=1)
features_test=features_test.drop('Total value sale', axis=1)

In [ ]:
print(features_train.shape, label_train.shape)

(58649, 13) (58649,)


In [ ]:
print(features_train.columns)

Index(['Customer', 'Sales Rep', 'Code Prod', 'Quantity', 'Units',
       'Total quantity', 'Value Sale', 'Tax', 'Channel', 'Brand', 'Category',
       'Unit price', 'NewDate'],
      dtype='object')


In [ ]:
cat_names = ['Customer', 'Sales Rep', 'Code Prod', 'Channel',  'Brand', 'Category','Units' ] 
num_names = [ 'NewDate', 'Quantity', 'Total quantity', 'Value Sale', 'Tax','Unit price'] 

In [ ]:
features_cat = features_train[cat_names]
features_num = features_train[num_names]

In [ ]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.attribute_names = feature_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_names)),
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

temp = num_pipeline.fit_transform(features_num)

In [ ]:
# handle_unknown = 'ignore' to handle item IDs that appear only in the test set
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_names)),
        ('cat_encoder', OneHotEncoder(sparse=False,handle_unknown = 'ignore')),
    ])

temp1 = cat_pipeline.fit_transform(features_train)

In [ ]:
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])
data_prepared = full_pipeline.fit_transform(features_train)

In [ ]:
print(data_prepared.shape, label_train.shape)

(58649, 3219) (58649,)


## Linear Regression

In [ ]:
start = time.time()

linear_regression = LinearRegression()
linear_regression.fit(data_prepared, label_train)

t=(time.time() - start)/60
print( 'Time in minutes:%5.2f'%t)

Time in minutes: 1.27


In [ ]:
predictions = linear_regression.predict(data_prepared)

mse = mean_squared_error(label_train, predictions)
rmse = np.sqrt(mse)
print('RMSE %5.4f'% rmse)

score = linear_regression.score(data_prepared, label_train)
print(' r^2 score %8.3f \n' % score)

RMSE 0.0000
 r^2 score    1.000 



In [ ]:
test_prepared = full_pipeline.transform(features_test)

test_prepared.shape

(16183, 3219)

In [ ]:
final_predictions = linear_regression.predict(test_prepared)

final_mse = mean_squared_error(label_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('RMSE %5.4f'%final_rmse)

score = linear_regression.score(test_prepared,label_test)
print(' r^2 score %8.3f \n' % score)

RMSE 2.0712
 r^2 score    1.000 



Business insight

In [ ]:
df = pd.DataFrame( columns=['prediction','label'] )
df['prediction']=pd.Series(final_predictions)
df['label']=label_test.reset_index()['Total value sale']
df

,prediction,label
0,224.851298,225.18
1,210.000000,210.00
2,360.000000,360.00
3,175.000000,175.00
4,292.000000,292.00
...,...,...
16178,-13.473716,0.01
16179,66.516284,80.00
16180,40.266284,53.75
16181,20.443383,34.30


In [ ]:
Ds=pd.DataFrame(columns=['P','L','D'] )
i=0
for index,row in df.iterrows():
    diff= round(row['prediction'],0)-row['label']
    if not diff>1:
        i+=1
        Ds=Ds.append({'P': row['prediction'],'L': row['label'],'D':diff },ignore_index=True )
print(i)

16113


In [ ]:
Ds.describe()

,P,L,D
count,16113.000000,16113.000000,16113.000000
mean,651.696972,652.008897,-0.299904
std,2029.370141,2029.324797,1.864076
min,-17688.290000,-17688.290000,-31.000000
25%,77.400000,77.930000,-0.240000
50%,201.000000,201.300000,0.000000
75%,527.310000,527.310000,0.100000
max,83568.000000,83568.000000,1.000000


In [ ]:
# Out of the test set (three last months, 6023 transactions)
# There were 348 predictions which were not perfect
# and you can further analyze them through the data frame Ds

## Decision Tree

In [ ]:
import time
start = time.time()

DTR= DecisionTreeRegressor()
DTR.fit(data_prepared, label_train)

t=(time.time() - start)/60
print( 'Time in minutes:%5.2f'%t)

Time in minutes: 0.45


In [ ]:
predictions = DTR.predict(data_prepared)

DTR_mse = mean_squared_error(label_train, predictions)
print('%5.4f'%DTR_mse)

DTR_rmse = np.sqrt(DTR_mse)
print('RMSE %5.4f'%DTR_rmse)

0.0000
RMSE 0.0001


In [ ]:
scores = cross_val_score(DTR, data_prepared, label_train, scoring="neg_mean_squared_error", cv=3)
DTR_rmse_scores = np.sqrt(-scores)
r2_scores = cross_val_score(DTR, data_prepared, label_train, scoring="r2", cv=3)

In [ ]:
print(DTR_rmse_scores)
print(r2_scores)

[  16.07324259 3548.95779502  277.14625879]
[0.99993757 0.45725354 0.97778836]


In [ ]:
test_prepared = full_pipeline.transform(features_test)

test_prepared.shape

(16183, 3219)

In [ ]:
final_predictions = DTR.predict(test_prepared)

final_mse = mean_squared_error(label_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('RMSE %5.4f'%final_rmse)

RMSE 130.0713


In [ ]:
score = DTR.score(test_prepared,label_test)
print(' r^2 score %8.3f \n' % score)

 r^2 score    0.996 



In [ ]:
#DTR = DTR.fit(data_prepared, label_train)
#tree.plot_tree(DTR) 
#plt.show()

In [ ]:
#DTR = DTR.fit(test_prepared, label_test)
#tree.plot_tree(DTR) 
#plt.show()

Business Insights

In [ ]:
df = pd.DataFrame( columns=['prediction','label'] )
df['prediction']=pd.Series(final_predictions)
df['label']=label_test.reset_index()['Total value sale']
df

,prediction,label
0,225.180000,225.18
1,210.000000,210.00
2,360.000000,360.00
3,175.000000,175.00
4,291.780000,292.00
...,...,...
16178,0.009994,0.01
16179,80.000000,80.00
16180,53.760000,53.75
16181,34.300000,34.30


In [ ]:
Ds=pd.DataFrame(columns=['P','L','D'] )
i=0
for index,row in df.iterrows():
    diff= round(row['prediction'],0)-row['label']
    if not diff>1:
        i+=1
        Ds=Ds.append({'P': row['prediction'],'L': row['label'],'D':diff },ignore_index=True )
print(i)

15800


In [ ]:
Ds.describe()

,P,L,D
count,15800.000000,15800.000000,15800.000000
mean,533.977006,535.721676,-1.748132
std,1531.105891,1592.140277,127.635970
min,-18028.350000,-17688.290000,-15410.000000
25%,76.500000,76.500000,-0.200000
50%,196.920000,196.890000,0.000000
75%,498.960000,498.960000,0.160000
max,68157.590000,83568.000000,1.000000


## Random Forest

In [ ]:
start = time.time()

forest = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42)
forest.fit(data_prepared, label_train)

t=(time.time() - start)/60
print( 'Time in minutes:%5.2f'%t)

Time in minutes: 3.45


In [ ]:
predictions = forest.predict(data_prepared)

forest_mse = mean_squared_error(label_train, predictions)
forest_rmse = np.sqrt(forest_mse)
print('RMSE %5.4f'%forest_rmse)

score = forest.score(data_prepared,label_train)
print(' r^2 score %8.3f \n' % score)

RMSE 769.0342
 r^2 score    0.937 



In [ ]:
final_predictions = forest.predict(test_prepared)

final_mse = mean_squared_error(label_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('RMSE %5.4f'%final_rmse)

RMSE 151.8953


In [ ]:
score = forest.score(test_prepared,label_test)
print(' r^2 score %8.3f \n' % score)

 r^2 score    0.994 



Business Insights

In [ ]:
df = pd.DataFrame( columns=['prediction','label'] )
df['prediction']=pd.Series(final_predictions)
df['label']=label_test.reset_index()['Total value sale']
df

,prediction,label
0,206.906140,225.18
1,204.670366,210.00
2,360.315953,360.00
3,183.747382,175.00
4,292.080095,292.00
...,...,...
16178,1.829805,0.01
16179,73.331037,80.00
16180,57.897120,53.75
16181,29.478857,34.30


In [ ]:
Ds=pd.DataFrame(columns=['P','L','D'] )
i=0
for index,row in df.iterrows():
    diff= round(row['prediction'],0)-row['label']
    if not diff>1:
        i+=1
        Ds=Ds.append({'P': row['prediction'],'L': row['label'],'D':diff },ignore_index=True )
print(i)

9748


In [ ]:
Ds.describe()

,P,L,D
count,9748.000000,9748.000000,9748.000000
mean,677.585915,684.519141,-6.937073
std,1913.667508,1996.581372,195.091370
min,-19975.440000,-17688.290000,-18837.000000
25%,87.235918,93.780000,-6.182500
50%,243.481438,243.680000,-2.600000
75%,610.155280,617.862500,-0.300000
max,64730.876000,83568.000000,1.000000
